In [1]:
#CUDA 11.8
#pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [2]:
import torch

print(torch.cuda.is_available()) 

True


In [3]:
print(torch.cuda.current_device())

torch.cuda.get_device_name(0)

0


'NVIDIA GeForce RTX 3050 Laptop GPU'

In [4]:
!git clone https://github.com/ultralytics/yolov5 

Cloning into 'yolov5'...


In [5]:
%cd yolov5
#%pip install -r requirements.txt

c:\yolov5_env\WeedyRice_Classification\yolov5


C:\Users\MSI 15\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Train dataset

In [6]:
# Train YOLOv5s Classification on dataset directory for 5 epochs

!python classify/train.py --model yolov5s-cls.pt --data ../dataset --epochs 5 --img 224 --cache

requirements: Ultralytics requirement ['setuptools>=70.0.0'] not found, attempting AutoUpdate...



   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.2 MB ? eta -:--:--
   ----------------- ---------------------- 0.5/1.2 MB 2.1 MB/s eta 0:00:01
   ---------------------------------- ----- 1.0/1.2 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 2.1 MB/s eta 0:00:00


  Attempting uninstall: setuptools

    Found existing installation: setuptools 57.4.0

    Uninstalling setuptools-57.4.0:

      Successfully uninstalled setuptools-57.4.0



requirements: AutoUpdate success ✅ 14.2s, installed 1 package: ['setuptools>=70.0.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



classify\train: model=yolov5s-cls.pt, data=../dataset, epochs=5, batch_size=64, imgsz=224, nosave=False, cache=ram, device=, workers=8, project=runs\train-cls, name=exp, exist_ok=False, pretrained=True, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, local_rank=-1
github: up to date with https://github.com/ultralytics/yolov5 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.
YOLOv5  v7.0-389-ge62a31b6 Python-3.10.0 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

TensorBoard: Start with 'tensorboard --logdir runs\train-cls', view at http://localhost:6006/
albumentations:  not found, install with `pip install albumentations` (recommended)

  0%|          

Test dataset

In [7]:
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --img 224 --source ../test  --name exp_images

tensor([[-1.89788,  1.14860]], device='cuda:0')
tensor([[-1.54935,  0.92175]], device='cuda:0')
tensor([[-3.17570,  2.14053]], device='cuda:0')
tensor([[ 2.08096, -2.54075]], device='cuda:0')
tensor([[ 1.59656, -1.04070]], device='cuda:0')
tensor([[ 1.52284, -1.09146]], device='cuda:0')


classify\predict: weights=['runs/train-cls/exp/weights/best.pt'], source=../test, data=data\coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs\predict-cls, name=exp_images, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-389-ge62a31b6 Python-3.10.0 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 117 layers, 4169250 parameters, 0 gradients, 10.4 GFLOPs
image 1/6 C:\yolov5_env\WeedyRice_Classification\test\1.jpg: 224x224 Weedy Rice 0.95, Cultivated Rice 0.05, 100.3ms
image 2/6 C:\yolov5_env\WeedyRice_Classification\test\2.png: 224x224 Weedy Rice 0.92, Cultivated Rice 0.08, 4.1ms
image 3/6 C:\yolov5_env\WeedyRice_Classification\test\3.png: 224x224 Weedy Rice 1.00, Cultivated Rice 0.00, 6.1ms
image 4/6 C:\yolov5_env\WeedyRice_Classification\test\4.jpg: 224x224 Cultivated Rice 0.99, Weedy Rice 0.01, 4.0ms
image 5/6 C:\yolov5

Convert Pytorch to ONNX

In [8]:
#before and during convert to onnx

import os
import torch
from torchsummary import summary
import onnxruntime as ort
import numpy as np
import torch.nn.functional as F
from torchvision import transforms

# Define the model file path
model_path = r'C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\weights\best.pt'

# Define the paths for the input and logits files
pytorch_input_path = r'C:\yolov5_env\WeedyRice_Classification\yolov5\pytorch_input.pt'
pytorch_logits_path = r'C:\yolov5_env\WeedyRice_Classification\yolov5\pytorch_logits.pt'


# Define the directory and file path for the ONNX model
onnx_model_dir = r'C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\weights'
onnx_model_path = os.path.join(onnx_model_dir, "model.onnx")

# Define preprocessing identical to `predict.py`
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Match input size
    transforms.ToTensor(),          # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Example mean/std
])

# Step 1: Load the model
try:
    model_data = torch.load(model_path, map_location=torch.device('cpu'))
    model = model_data['model']
    model = model.float().to('cpu')  # Ensure model is in full precision and on CPU
    model.eval()  # Set model to evaluation mode
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Step 2: Inspect Metadata
print("\n--- Model Metadata ---")
if isinstance(model_data, dict):
    print(f"Keys in model file: {model_data.keys()}")

# Step 3: Model Summary
print("\n--- Model Summary ---")
try:
    dummy_input = torch.randn(1, 3, 224, 224).to(torch.device('cpu'))  # Ensure input is on CPU
    model.to('cpu')  # Ensure model is also on CPU
    summary(model, input_size=(3, 224, 224), device="cpu")  # Specify device explicitly
except Exception as e:
    print(f"Unable to summarize model: {e}")


# Step 4: Load Preprocessed Input and Logits
print("\n--- Loading PyTorch Preprocessed Input and Logits ---")
try:
    pytorch_input = torch.load(pytorch_input_path).to('cpu')  # Load and move to CPU
    pytorch_logits = torch.load(pytorch_logits_path).to('cpu')  # Load and move to CPU
    print("Preprocessed input and logits loaded successfully!")
except Exception as e:
    print(f"Error loading preprocessed input or logits: {e}")
    exit()

# Step 5: Export PyTorch Model to ONNX
try:
    torch.onnx.export(
        model,
        pytorch_input,  # Use preprocessed input
        onnx_model_path,
        export_params=True,
        opset_version=11,
        input_names=["input"],
        output_names=["output"]
    )
    print(f"Model exported to ONNX successfully at {onnx_model_path}!")
except Exception as e:
    print(f"Error exporting PyTorch model to ONNX: {e}")

# Step 6: Test ONNX Model
try:
    session = ort.InferenceSession(onnx_model_path)
    onnx_input = pytorch_input.numpy()
    onnx_output = session.run(None, {session.get_inputs()[0].name: onnx_input})
    onnx_logits = onnx_output[0]
    print("ONNX Model tested successfully!")
except Exception as e:
    print(f"Error testing ONNX model: {e}")
    exit()

# Step 7: Compare Outputs
print("\n--- Comparing PyTorch and ONNX Outputs ---")
try:
    # Compare Logits
    pytorch_logits_np = pytorch_logits.numpy()
    onnx_logits_np = onnx_logits
    logit_diff = np.abs(pytorch_logits_np - onnx_logits_np)
    print("Logit Differences:", logit_diff)

    # Compare Probabilities
    pytorch_probabilities = F.softmax(torch.tensor(pytorch_logits_np), dim=1).numpy()
    onnx_probabilities = F.softmax(torch.tensor(onnx_logits_np), dim=1).numpy()
    probability_diff = np.abs(pytorch_probabilities - onnx_probabilities)
    print("Probability Differences:", probability_diff)

    # Predicted Classes
    pytorch_pred_class = np.argmax(pytorch_probabilities, axis=1)
    onnx_pred_class = np.argmax(onnx_probabilities, axis=1)
    print("PyTorch Predicted Class:", pytorch_pred_class)
    print("ONNX Predicted Class:", onnx_pred_class)
    print("Do predicted classes match?", np.array_equal(pytorch_pred_class, onnx_pred_class))

    # Save intermediate outputs for further debugging if needed
    np.save("onnx_logits.npy", onnx_logits_np)
    np.save("pytorch_logits.npy", pytorch_logits_np)
except Exception as e:
    print(f"Error comparing outputs: {e}")

print("\n--- Analysis Complete ---")


C:\Users\MSI 15\AppData\Local\Temp\ipykernel_16856\2129334730.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_data = torch.load(model_path, map_location=torch.dev

Model loaded successfully!

--- Model Metadata ---
Keys in model file: dict_keys(['epoch', 'best_fitness', 'model', 'ema', 'updates', 'optimizer', 'opt', 'git', 'date'])

--- Model Summary ---
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]           3,456
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
              Conv-4         [-1, 32, 112, 112]               0
            Conv2d-5           [-1, 64, 56, 56]          18,432
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              SiLU-7           [-1, 64, 56, 56]               0
              Conv-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 32, 56, 56]           2,048
      BatchNorm2d-10           [-1, 32, 56, 56]              64
             SiLU-11           [-1, 32

c:\yolov5_env\WeedyRice_Classification\yolov5\models\yolo.py:167: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if profile:
c:\yolov5_env\WeedyRice_Classification\yolov5\models\yolo.py:171: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if visualize:


Model exported to ONNX successfully at C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\weights\model.onnx!
ONNX Model tested successfully!

--- Comparing PyTorch and ONNX Outputs ---
Logit Differences: [[  0.0014409   0.0011091]]
Probability Differences: [[ 0.00016224  0.00016229]]
PyTorch Predicted Class: [0]
ONNX Predicted Class: [0]
Do predicted classes match? True

--- Analysis Complete ---


Check accuracy of ONNX

In [9]:
#after convert to onnx

import torch
import os
import cv2
import numpy as np
import onnxruntime as ort
import torch.nn.functional as F


# Define paths
onnx_model_dir = r"C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\weights"  # ONNX model directory
onnx_model_path = os.path.join(onnx_model_dir, "model.onnx")  # Path to the ONNX model

# Ensure the directory exists
if not os.path.exists(onnx_model_path):
    raise FileNotFoundError(f"ONNX model not found at {onnx_model_path}")

# Define paths
test_images_path = r"C:\yolov5_env\WeedyRice_Classification\test"  # Test images folder
class_names = ["Cultivated Rice", "Weedy Rice"]  # Class names

# Load ONNX model
print("Loading ONNX model...")
session = ort.InferenceSession(onnx_model_path)

# Preprocessing function
def preprocess_image(image_path, img_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    img = img / 255.0  # Normalize to [0, 1]
    img = (img - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])  # Normalize with mean/std
    img = np.transpose(img, (2, 0, 1))  # HWC to CHW
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img.astype(np.float32)


# Test ONNX model with test images
print("Testing ONNX model with test images...")
results = []
for img_file in os.listdir(test_images_path):
    img_path = os.path.join(test_images_path, img_file)
    if img_file.endswith((".jpg", ".png", ".jpeg")):
        print(f"\nProcessing image: {img_file}")

        # Preprocess image
        input_tensor = preprocess_image(img_path)

        # Run ONNX inference
        onnx_output = session.run(None, {session.get_inputs()[0].name: input_tensor})
        logits = onnx_output[0]

        # Post-process logits to probabilities
        probabilities = F.softmax(torch.tensor(logits), dim=1).numpy()[0]
        predicted_class = np.argmax(probabilities)
        predicted_class_name = class_names[predicted_class]

        # Save result
        results.append({
            "image": img_file,
            "logits": logits[0],
            "probabilities": probabilities,
            "predicted_class": predicted_class_name,
            "confidence": probabilities[predicted_class]
        })

        # Print results
        print(f"Logits: {logits[0]}")
        print(f"Probabilities: {probabilities}")
        print(f"Predicted Class: {predicted_class_name} ({probabilities[predicted_class]:.2f})")

# Summary of results
print("\n--- Summary of ONNX Model Predictions ---")
for res in results:
    print(f"Image: {res['image']}")
    print(f"  Predicted Class: {res['predicted_class']} (Confidence: {res['confidence']:.2f})")
    print(f"  Logits: {res['logits']}")
    print(f"  Probabilities: {res['probabilities']}")


Loading ONNX model...
Testing ONNX model with test images...

Processing image: 1.jpg
Logits: [    -1.0871      0.7644]
Probabilities: [     0.1357      0.8643]
Predicted Class: Weedy Rice (0.86)

Processing image: 2.png
Logits: [    -1.5106      1.2516]
Probabilities: [     0.0594      0.9406]
Predicted Class: Weedy Rice (0.94)

Processing image: 3.png
Logits: [    -2.5305       1.793]
Probabilities: [    0.01308     0.98692]
Predicted Class: Weedy Rice (0.99)

Processing image: 4.jpg
Logits: [     3.2403     -3.1878]
Probabilities: [    0.99839    0.001613]
Predicted Class: Cultivated Rice (1.00)

Processing image: 5.jpg
Logits: [     1.2091    -0.79328]
Probabilities: [    0.88105     0.11895]
Predicted Class: Cultivated Rice (0.88)

Processing image: 6.jpg
Logits: [     1.2447    -0.92001]
Probabilities: [    0.89704     0.10296]
Predicted Class: Cultivated Rice (0.90)

--- Summary of ONNX Model Predictions ---
Image: 1.jpg
  Predicted Class: Weedy Rice (Confidence: 0.86)
  Logits:

Convert ONNX to Keras

In [10]:
#convert to keras

import onnx
from onnx_tf.backend import prepare
import tensorflow as tf
import os

# Define paths
onnx_model_dir = r"C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\weights"  # ONNX model directory
onnx_model_path = os.path.join(onnx_model_dir, "model.onnx")  # Path to the ONNX model

# Ensure the directory exists
if not os.path.exists(onnx_model_path):
    raise FileNotFoundError(f"ONNX model not found at {onnx_model_path}")


tf_saved_model_dir = r"C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\saved_model"
keras_saved_model_dir = r"C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\saved_model"

# Step 1: Convert ONNX to TensorFlow SavedModel
print("Loading ONNX model...")
onnx_model = onnx.load(onnx_model_path)
print("Converting ONNX to TensorFlow SavedModel...")
os.makedirs(tf_saved_model_dir, exist_ok=True)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_saved_model_dir)
print(f"TensorFlow SavedModel saved at {tf_saved_model_dir}!")

# Step 2: Load TensorFlow SavedModel
print("Loading TensorFlow SavedModel...")
loaded_model = tf.saved_model.load(tf_saved_model_dir)

# Inspect available outputs
print("Inspecting model signature...")
signatures = loaded_model.signatures["serving_default"]
print("Inputs:", signatures.structured_input_signature)
print("Outputs:", signatures.structured_outputs)

# Extract the correct output key
output_key = list(signatures.structured_outputs.keys())[0]  # Use the first available output key
print(f"Using output key: {output_key}")

# Define the wrapped model class
class WrappedModel(tf.keras.Model):
    def __init__(self, saved_model, output_key):
        super(WrappedModel, self).__init__()
        self.saved_model = saved_model
        self.model_function = saved_model.signatures["serving_default"]
        self.output_key = output_key

    def call(self, inputs):
        # Ensure inputs are properly passed
        outputs = self.model_function(inputs)
        return outputs[self.output_key]
    
    def get_config(self):
        # Define configuration for serialization
        return {"output_key": self.output_key}

    @classmethod
    def from_config(cls, config, custom_objects=None):
        # Recreate the model from the config (requires the saved_model to be passed externally)
        raise NotImplementedError("Loading this model requires the TensorFlow SavedModel.")

print("Converting to Keras model...")
keras_model = WrappedModel(loaded_model, output_key)

# Define a concrete input shape
sample_input = tf.random.uniform([1, 3, 224, 224])  # Match input size as per ONNX model
_ = keras_model(sample_input)  # Build the model by calling it with a sample input

# Save as Keras SavedModel
print("Saving as Keras SavedModel...")
os.makedirs(keras_saved_model_dir, exist_ok=True)
tf.keras.models.save_model(keras_model, keras_saved_model_dir, save_format="tf")
print(f"Keras SavedModel saved at {keras_saved_model_dir}!")


Loading ONNX model...
Converting ONNX to TensorFlow SavedModel...
INFO:tensorflow:Assets written to: C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\saved_model\assets


INFO:tensorflow:Assets written to: C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\saved_model\assets


TensorFlow SavedModel saved at C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\saved_model!
Loading TensorFlow SavedModel...
Inspecting model signature...
Inputs: ((), {'input': TensorSpec(shape=(1, 3, 224, 224), dtype=tf.float32, name='input')})
Outputs: {'output': TensorSpec(shape=(1, 2), dtype=tf.float32, name='output')}
Using output key: output
Converting to Keras model...
Saving as Keras SavedModel...
INFO:tensorflow:Assets written to: C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\saved_model\assets


INFO:tensorflow:Assets written to: C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\saved_model\assets


Keras SavedModel saved at C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\saved_model!


Check accuracy of saved_model.pb (keras)

In [11]:
#check keras model accuracy

import tensorflow as tf
import numpy as np
import cv2
import os

# Path to SavedModel
saved_model_dir = r"C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\saved_model"
test_images_path = r"C:\yolov5_env\WeedyRice_Classification\test"  # Test images folder
class_names = ["Cultivated Rice", "Weedy Rice"]  # Class names

# Load the SavedModel
print("Loading TensorFlow SavedModel...")
model = tf.saved_model.load(saved_model_dir)
infer = model.signatures["serving_default"]

# Inspect available outputs
print("Inspecting model signature...")
print("Inputs:", infer.structured_input_signature)
print("Outputs:", infer.structured_outputs)

# Dynamically determine the output key
output_key = list(infer.structured_outputs.keys())[0]  # Pick the first available output key
print(f"Using output key: {output_key}")

# Preprocessing function
def preprocess_image(image_path, img_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    img = img / 255.0  # Normalize to [0, 1]
    img = (img - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])  # Normalize
    img = np.transpose(img, (2, 0, 1))  # Change to channels-first format
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img.astype(np.float32)

# Test images
print("Testing SavedModel with test images...")
for img_file in os.listdir(test_images_path):
    img_path = os.path.join(test_images_path, img_file)
    if img_file.endswith((".jpg", ".png", ".jpeg")):
        print(f"\nProcessing image: {img_file}")
        
        # Preprocess image
        input_tensor = preprocess_image(img_path)

        # Run inference
        predictions = infer(tf.convert_to_tensor(input_tensor))[output_key].numpy()[0]

        # Post-process logits to probabilities
        probabilities = tf.nn.softmax(predictions).numpy()
        predicted_class = np.argmax(probabilities)
        predicted_class_name = class_names[predicted_class]

        print(f"Logits: {predictions}")
        print(f"Probabilities: {probabilities}")
        print(f"Predicted Class: {predicted_class_name} ({probabilities[predicted_class]:.2f})")


Loading TensorFlow SavedModel...
Inspecting model signature...
Inputs: ((), {'input_1': TensorSpec(shape=(None, 3, 224, 224), dtype=tf.float32, name='input_1')})
Outputs: {'output_1': TensorSpec(shape=(1, 2), dtype=tf.float32, name='output_1')}
Using output key: output_1
Testing SavedModel with test images...

Processing image: 1.jpg
Logits: [    -1.0871      0.7644]
Probabilities: [     0.1357      0.8643]
Predicted Class: Weedy Rice (0.86)

Processing image: 2.png
Logits: [    -1.5106      1.2516]
Probabilities: [     0.0594      0.9406]
Predicted Class: Weedy Rice (0.94)

Processing image: 3.png
Logits: [    -2.5305       1.793]
Probabilities: [    0.01308     0.98692]
Predicted Class: Weedy Rice (0.99)

Processing image: 4.jpg
Logits: [     3.2403     -3.1878]
Probabilities: [    0.99839    0.001613]
Predicted Class: Cultivated Rice (1.00)

Processing image: 5.jpg
Logits: [     1.2091    -0.79328]
Probabilities: [    0.88105     0.11895]
Predicted Class: Cultivated Rice (0.88)

Pro

Convert Keras to tflite

In [12]:
import tensorflow as tf

# Path to your SavedModel directory and output TFLite file
saved_model_dir = r"C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\saved_model"
output_file = r"C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\weights\model.tflite"

# Convert the SavedModel to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the TFLite model
with open(output_file, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved to: {output_file}")


TFLite model saved to: C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\weights\model.tflite


Check tflite accuarcy

In [13]:
import tensorflow as tf
import numpy as np
import cv2
import os

# Paths
tflite_model_path = r"C:\yolov5_env\WeedyRice_Classification\yolov5\runs\train-cls\exp\weights\model.tflite"
test_images_path = r"C:\yolov5_env\WeedyRice_Classification\test"  # Folder with test images
class_names = ["Cultivated Rice", "Weedy Rice"]  # Class names
ground_truth_labels = {"1.jpg": 1, "2.png": 1, "3.png": 1, "4.jpg": 0, "5.jpg": 0, "6.jpg": 0}  # Example labels

# Load the TFLite model
print("Loading TFLite model...")
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Verify the input shape
expected_input_shape = input_details[0]['shape']  # Shape of input tensor
print(f"Expected input shape: {expected_input_shape}")

# Preprocessing function
def preprocess_image(image_path, img_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, img_size)  # Resize to model input size
    img = img / 255.0  # Normalize to [0, 1]
    img = (img - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])  # Normalize with mean/std
    img = np.transpose(img, (2, 0, 1))  # Convert HWC to CHW
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img.astype(np.float32)

# Evaluate TFLite model
print("Evaluating TFLite model...")
correct_predictions = 0
total_predictions = 0

for img_file in os.listdir(test_images_path):
    img_path = os.path.join(test_images_path, img_file)
    if img_file.endswith((".jpg", ".png", ".jpeg")):
        print(f"\nProcessing image: {img_file}")

        # Preprocess the image
        input_data = preprocess_image(img_path)

        # Check the input shape
        print(f"Input tensor shape: {input_data.shape}")
        if input_data.shape != tuple(expected_input_shape):
            raise ValueError(f"Input shape mismatch. Expected: {expected_input_shape}, Got: {input_data.shape}")

        # Run inference
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        predictions = interpreter.get_tensor(output_details[0]['index'])[0]

        # Post-process predictions
        probabilities = tf.nn.softmax(predictions).numpy()
        predicted_class = np.argmax(probabilities)
        predicted_class_name = class_names[predicted_class]

        # Compare with ground truth
        ground_truth = ground_truth_labels[img_file]
        is_correct = predicted_class == ground_truth
        correct_predictions += int(is_correct)
        total_predictions += 1

        print(f"Logits: {predictions}")
        print(f"Probabilities: {probabilities}")
        print(f"Predicted Class: {predicted_class_name} ({probabilities[predicted_class]:.2f})")
        print(f"Ground Truth: {class_names[ground_truth]}")
        print(f"Correct: {is_correct}")

# Calculate accuracy
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
print(f"\n--- Evaluation Complete ---")
print(f"Accuracy: {accuracy:.2%} ({correct_predictions}/{total_predictions})")


Loading TFLite model...
Expected input shape: [  1   3 224 224]
Evaluating TFLite model...

Processing image: 1.jpg
Input tensor shape: (1, 3, 224, 224)
Logits: [    -1.0871      0.7644]
Probabilities: [     0.1357      0.8643]
Predicted Class: Weedy Rice (0.86)
Ground Truth: Weedy Rice
Correct: True

Processing image: 2.png
Input tensor shape: (1, 3, 224, 224)
Logits: [    -1.5106      1.2516]
Probabilities: [     0.0594      0.9406]
Predicted Class: Weedy Rice (0.94)
Ground Truth: Weedy Rice
Correct: True

Processing image: 3.png
Input tensor shape: (1, 3, 224, 224)
Logits: [    -2.5305       1.793]
Probabilities: [    0.01308     0.98692]
Predicted Class: Weedy Rice (0.99)
Ground Truth: Weedy Rice
Correct: True

Processing image: 4.jpg
Input tensor shape: (1, 3, 224, 224)
Logits: [     3.2402     -3.1878]
Probabilities: [    0.99839    0.001613]
Predicted Class: Cultivated Rice (1.00)
Ground Truth: Cultivated Rice
Correct: True

Processing image: 5.jpg
Input tensor shape: (1, 3, 224